Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

# LINMA1702 Optimisation d'un réseau de distribution d'eau

## I. Analyse d'un réseau existant

### I.1 Que représentent les expressions $A^Th$ et $Af$ ?

Soit h le vecteur contenant les hauteurs en chaque point, f le vecteur contenant les débits circulant dans les conduites et A la matrice d'incidence.

L'expression $A^Th$ est le vecteur contenant les dénivelés de chaque conduite.

L'expression $Af$ est le vecteur contenant les débits nets en chaque point.

### I.2 

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [425]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np

In [426]:
def readFile(file_name):
    """
    Cette fonction lit un fichier formatté au type utilise dans les exemples du
    projet
    @pre                : Le fichier file_name est organise de la meme maniere
                          que les exemples donnes sur Moodle.
    
    @param file_name    : contient un String avec le nom du fichier a lire.
    
    @return n_points    : Contient un 'int' avec le nombre de points
    @return n_conduites : Contient un 'int' avec le nombre de conduites
    @return X           : Contient un 'numpy.array' avec les abscisses des points.
    @return Y           : Contient un 'numpy.array' avec les ordonnees des points.
    @return Z           : Contient un 'numpy.array' avec la hauteur des points.
    @return A           : Contient un 'numpy.array' avec la matrice d'incidence
                          des conduites.
    @return alpha       : Contient un 'float' avecla valeur de alpha.
    @return R           : Contient un 'float' avec le rayon des conduites.
    @return pa          : Contient un 'numpy.array' avec les points
                          d'approvisionnement.
    @return da_max      : Contient un 'numpy.array' avec les debits maximaux
                          extractibles.
    @return C           : Contient un 'numpy.array' avec les couts d'extraction.
    @return pc          : Contient un 'numpy.array' avec les points de
                          consommation.
    @return dc_min      : Contient un 'numpy.array' avec les debits minimaux
                          consommables.
    @return dc_max      : Contient un 'numpy.array' avec les debits maximaux
                          consommables.
    @return Pf          : Contient un 'float' avec le prix facture.
    @return pi          : Contient un 'numpy.array' avec les points
                          intermediaires.
    @return unit        : Contient un 'String' avec l'unite des coordonnes des
                          points.
    """
    with open(file_name,'r') as f:
        n_points    = (int)(f.readline().split()[0])
        n_conduites = (int)(f.readline().split()[0])
        f.readline()
        unit        = (f.readline().split(" = ")[1])
        
        coord       = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        X           = coord[:,0]
        Y           = coord[:,1]
        Z           = coord[:,2]
        
        f.readline()
        f.readline()
        A           = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        f.readline()
        
        UL_alpha    = f.readline().split(" = ")[1]
        UL_alpha    = UL_alpha.split()[0]
        UL2_alpha   = np.array(UL_alpha.split("^"))
        if(len(UL2_alpha) == 1):
            alpha   = (float)(UL2_alpha)
        else:
            alpha   = (float)(UL2_alpha[0]) ** (float)(UL2_alpha[1])
        
        R           = (float)(f.readline().split()[4])
        f.readline()
        
        UL_pa       = f.readline().split(" = [")[1]
        UL_pa       = UL_pa.split("]")[0]
        pa          = np.array(list(int(w) for w in UL_pa.split()[:]))
        
        UL_da_max   = f.readline().split(" = [")[1]
        UL_da_max   = UL_da_max.split("]")[0]
        da_max      = np.array(list(float(w) for w in UL_da_max.split()[:]))
        
        UL_C        = f.readline().split(" = [")[1]
        UL_C        = UL_C.split("]")[0]
        C           = np.array(list(float(w) for w in UL_C.split()[:]))
        
        f.readline()
        
        UL_pc       = f.readline().split(" = [")[1]
        UL_pc       = UL_pc.split("]")[0]
        pc          = np.array(list(int(w) for w in UL_pc.split()[:]))
        
        UL_dc_min   = f.readline().split(" = [")[1]
        UL_dc_min   = UL_dc_min.split("]")[0]
        dc_min      = np.array(list(float(w) for w in UL_dc_min.split()[:]))
        
        UL_dc_max   = f.readline().split(" = [")[1]
        UL_dc_max   = UL_dc_max.split("]")[0]
        dc_max      = np.array(list(float(w) for w in UL_dc_max.split()[:]))
        
        Pf          = (float)(f.readline().split()[3])
        
        f.readline()
        
        UL_pi       = f.readline().split("[")[1]
        UL_pi       = UL_pi.split("]")[0]
        pi          = np.array(list(int(w) for w in UL_pi.split()[:]))
        
    return (n_points, n_conduites, X, Y, Z, A, alpha, R, pa, da_max, C, pc, dc_min, dc_max, Pf, pi, unit)

In [427]:
def readSecondFile(filename):
    """
    Cette fonction permet de récupérer les données stockées dans le fichier 'partie2'
    @pre : filename est un String pointant vers un fichier.
    @pre : le fichier 'filename' est formaté correctement.
    @pre : le cout de construction d'un chateau d'eau est exprime en millions.
    
    @return Cchateau contient le cout de construction d'un chateau d'eau (€/m).
    @return Hmax     contient la hauteur max. d'un chateau d'eau (metre).
    @return Btot     contient le budget total de construction des chateaux.
    @return Ccc      contient le cout de construction d'une conduite.
    """
    with open(filename,'r') as f:
        f.readline()
        f.readline()
        Cchateau = (float)(f.readline().split()[7])*1000000
        
        UL_Hmax  = (f.readline().split()[6]).split(",")
        if(len(UL_Hmax)==2):
            Hmax = (float)(UL_Hmax[0]) + (float)(UL_Hmax[1])/(10**(len(UL_Hmax[1])))
        else:
            Hmax = (float)(UL_Hmax)
        Btot     = (float)(f.readline().split()[7])
        f.readline()
        Ccc      = (float)(f.readline().split()[6])
    return (Cchateau, Hmax, Btot, Ccc)

In [428]:
#Fonction retournant [f,xC,xA] avec f la valeur de la fonction objectif maximale 
#xC un vecteur des valeurs optimales des débits aux points de consommation
#xA un vecteur des valeurs optimales des débits aux points d'approvisionnement
#Soit Pf le prix facturé
#Soit C le cout
#Soit DCmin et DCmax les valeurs minimales et maximales du débit aux points de consommation
#Soit DAmax les valeurs maximales du débit extractibles aux sources
def Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha):
   
    h = abs(A.T@Z)      # le denivele de chaque conduite entre le pt de depart et d'arrivee.
    x=abs(A.T@X)        # la difference de x de chaque conduite entre le pt de depart et d'arrivee.
    y=abs(A.T@Y)        # la difference de y de chaque conduite entre le pt de depart et d'arrivee.
    L=np.sqrt(h*h+x*x+y*y)       
    
    A1     = np.concatenate((A[np.array(Pc)-1],-1* A[np.array(Pa)-1]))   # Tous les debits positifs
    A3     = np.concatenate((A[np.array(Pc)-1], A[np.array(Pa)-1]))      # On laisse les debits aux points d'approvisionnement 
                                                                         # negatifs
    DebMax = alpha*R*R*h/L*A1   # Debit max, juste sur les points pas 
                                                                         # intermediaires
    
    A2     = A[Pc-1]                                           # Uniquement les points de consommation.
    DebMin = -1*alpha*R*R*h/L*A2 # Debit min
    
    #Sum    = np.sum(np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A3), axis=0)    
    #Aub    = np.concatenate((np.concatenate((DebMax,DebMin)),np.reshape(Sum, (1,np.shape(DebMax)[1]))))
    Aub = np.concatenate((DebMax,DebMin))
    Argent = np.concatenate((Pf,C))
    Eye=np.eye(len(Argent))*Argent
    #print(Eye)
    c=-1*np.matmul(Argent,A3)*(alpha*R*R*h/L)    
    print(c)
    A5=A[np.array(Pi)-1] #On ne prend que les points 
    Aeq=A5*alpha*R*R*h/L #Points intemédiaires ont un débit resultant nul
    beq=np.zeros(len(Pi))
    #c=np.sum(alpha*R*R*h/L*A3*np.outer(np.concatenate((Pf,C)),np.ones(len(A3[0]))),axis=0)
    
    #b      = np.concatenate((np.concatenate((Dcmax,np.array(Damax))),np.concatenate((-1*np.array(Dcmin),[0]))))
    b      = np.concatenate((np.concatenate((Dcmax, Damax)), -1*Dcmin))
    
    bounds = np.concatenate((np.zeros((nConduites,1)),np.ones((nConduites,1))),axis=1) # limites (vecteur de (0,1) sur le
                                                                                       # nombre de conduites)
    res    = linprog(c, A_ub = Aub, b_ub = b,A_eq=Aeq, b_eq=beq,bounds=bounds,options={'tol': 1e-8})
    DebC=(A2*alpha*R*R*h/L)@res.x
    A4=A[Pa-1]
    DebA=(A4*alpha*R*R*h/L)@res.x
    Prix=Pf.T@DebC
    Couts=C.T@DebA
    print("This is res")
    #print(res)
    return (-1*res.fun,DebC,DebA,Prix,Couts)#modifier le return

In [1]:
(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi, unit)=readFile("Mini-exemple-avec-solution/Data-mini-exemple-avec-solution.txt")
print(Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Dcmin)),Pi,alpha))

[  594.17617249 -4182.82750547 -2427.26857345   444.46868816
   579.6483884   1108.4424655     -0.         -3811.16560356
 -2470.39546141    -0.            -0.           121.96020175
   985.01280798  -581.90027837    -0.            -0.
    -0.            -0.            -0.            -0.
    -0.            -0.            -0.            -0.
    -0.            -0.            -0.            -0.
    -0.            -0.            -0.         -5048.19806534
    -0.            -0.         -5680.6628526   -967.60837482
    -0.         -3344.1790651   3472.13558932]
This is res
(6500.7015612316245, array([4000.        , 2000.        ,  646.55586486, 3000.        ,
        211.58922262]), array([-5858.14508747, -4000.        ]), 8872.330578726376, -2371.6290174947503)


# II. Améliorations du réseau

In [430]:
def Ameliorations(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi): 
    h = abs(A.T@Z)      # le denivele de chaque conduite entre le pt de depart et d'arrivee.
    x = abs(A.T@X)        # la difference de x de chaque conduite entre le pt de depart et d'arrivee.
    y = abs(A.T@Y)        # la difference de y de chaque conduite entre le pt de depart et d'arrivee.
    L=np.sqrt(h*h+x*x+y*y)
    Ac=A[Pc-1]#On prend les lignes de la matrice d'incidence relatives aux points de consommation
    Aa=A[Pa-1]#On prend les lignes de la matrice d'incidence relatives aux points d'approvisionnement
    Ai=A[Pi-1]#On prend les lignes de la matrice d'incidence relatives aux points intermediaires
    debit=alpha*R*R*h/L #Formule de debit
    Argent = np.concatenate([Pf,C,Pf/2])
    grandA=np.concatenate([np.concatenate([Ac,Aa,Ac],axis=0)*debit,np.concatenate([Ac,Aa,Ac],axis=0)*debit],axis=1)
    c=-1*np.matmul(Argent,grandA)#fonction objectif
    A1=np.concatenate([Ac*debit,np.zeros((len(Ac),len(Ac[0])))],axis=1)#Dc<Dcmax et -Dc<-Dcmin
    Atrop=np.concatenate([np.zeros((len(Ac),len(Ac[0]))),Ac*debit],axis=1)#Dctrop<0,25*Dcmax
    Aapp=np.concatenate([-1*Aa*debit,-1*Aa*debit],axis=1)#Da<Damax
    theta=np.concatenate([np.eye(nConduites),np.eye(nConduites)],axis=1)#theta1+theta2<1
    Aub=np.concatenate([A1,-1*A1,Atrop,Aapp,theta]) #Concatenate de la grande matrice Aub
    b=np.concatenate([Dcmax,-1*Dcmin,0.25*Dcmax,Damax,np.ones(nConduites)])#Concatenate du grand vecteur de contraintes
    Aeq=np.concatenate([Ai*debit,Ai*debit],axis=1)#Di=0
    beq=np.zeros(len(Pi))#Di=0
    Bounds=(0,None)#theta>0
    res    = linprog(c, A_ub = Aub, b_ub = b,A_eq=Aeq, b_eq=beq,bounds=Bounds,options={'tol': 1e-8})#Resolution
    theta=res.x[0:nConduites]+res.x[nConduites:2*nConduites]
    print(theta)
    DebitCok=(Ac*debit)@res.x[0:nConduites]
    print(DebitCok)
    DebitCtrop=(Ac*debit)@res.x[nConduites:2*nConduites]
    print(DebitCtrop)
    DebitC=(Ac*debit)@theta
    DebitA=(-1*Aa*debit)@theta
    DebitI=(Ai*debit)@theta
    fonction=Pf.T@DebitCok+(Pf/2).T@DebitCtrop-C@DebitA
    return (-1*res.fun,DebitC,DebitA,DebitI,fonction)

In [431]:
print(Ameliorations(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Dcmin)),Pi))

(16, 78)
[0.84150126 0.21516546 0.98876574 1.         0.82318926 0.27065004
 0.         0.59037057 0.         0.         0.66345179 0.
 0.         1.         0.12441653 0.76515308 0.         0.
 0.         0.         0.16278679 1.         0.         0.
 0.10029054 0.         0.33604921 1.         0.33988275 0.
 0.32012399 0.53484431 0.         1.         0.11882416 0.
 0.         0.05694381 0.        ]
[4000. 2000.  200. 3000.  200.]
[1000.          500.          446.55586486  750.           11.58922262]
(13424.366850594812, array([5000.        , 2500.        ,  646.55586486, 3750.        ,
        211.58922262]), array([7108.14508747, 5000.        ]), array([-1.98437937e-14,  4.54747351e-13,  0.00000000e+00, -4.54747351e-13,
        0.00000000e+00,  4.54747351e-13,  0.00000000e+00,  1.13686838e-13,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -5.68434189e-14, -1.73968872e-15,  0.00000000e+00]), 6757.036271868437)


# III. Conception d'un réseau optimal